# API Center ❤️ all APIs

## Import from Amazon API Gateway
![image](../../images/import-from-aws.png)

Playground to experiment importing APIs from Amazon API Gateway. We start by creating an API Center instance using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) to export the APIs and the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) to import the APIs into API Center.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to:
- Deploy Azure API Center using Bicep.
- Configure the API Center metadata scheme for APIs, Environments and Deployments.
- Understand the Azure CLI commands to manage Azure API Center. [Full list of commands available here](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest).
- Register APIs using the OpenAPI spec exported from AWS API Gateway.
- Create deployments for each AWS API Gateway stage. 
- Search and discover the APIs registered in API Center.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)
- Install or update to the latest [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html)
- [Setup](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-quickstart.html) the AWS CLI to interact with AWS services

### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

# the following metadata will be created during the deployment
$metadata = @(
  @{
    "name" = 'createdDate'
    "schema" = '{"title":"createdDate","description":"Date when the API was created","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      },
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'rootResourceId'
    "schema" = '{"title":"rootResourceId","description":"API Gateway resource id","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'dataClassification'
    "schema" = '{"title":"dataClassification","description":"Classification of the Data that will transit in the API","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'department'
    "schema" = '{"title":"department","description":"Department associated with the API","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  }
)

# the following environment(s) will be created during the deployment
$environments = @(
  @{
    "title" = 'production'
    "kind" = 'production'
    "description" = 'Production Stage'
    "server" = @{
        "type" = 'AWS API Gateway' 
        "managementPortalUri" = 'https://console.aws.amazon.com/apigateway/'
    }
    "onboarding" = @{
        "developerPortalUri" = ''
        "instructions" = ''
    }
    "customProperties" = @{ 
    }
  },
  @{
    "title" = 'development'
    "kind" = 'development'
    "description" = 'Development Stage'
    "server" = @{
        "type" = 'AWS API Gateway' 
        "managementPortalUri" = 'https://console.aws.amazon.com/apigateway/'
    }
    "onboarding" = @{
        "developerPortalUri" = ''
        "instructions" = ''
    }
    "customProperties" = @{ 
    }
  }
)

Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Export the APIs from AWS API Gateway and Register in API Center

We will start to list all the REST APIs, then for each API we will export the OpenAPI spec, update the metadata and then create deployments for each environment.

Note: this script supports REST APIs. For HTTP APIs you should adapt the script to use the [equivalent commands](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/apigatewayv2/index.html).

In [ ]:

# comment one of the following lines to use the real AWS API Gateway or the local file
#$restAPIsJson = aws apigateway get-rest-apis --output json
$restAPIsJson = Get-Content -Raw 'get-rest-apis.json'

$restAPIs = $restAPIsJson | ConvertFrom-Json
foreach ($restAPI in $restAPIs.items) {
    $restAPIId = $restAPI.name

    # comment one of the following lines to use the real AWS API Gateway or the local file
    #$restAPIJson = aws apigateway get-rest-api --rest-api-id $restAPIId --output json
    $restAPIJson = Get-Content -Raw 'get-rest-api.json' 

    $restAPI = $restAPIJson | ConvertFrom-Json

    # uncomment the following line to export from AWS API Gateway instead of using the local file
    # aws apigateway get-export --rest-api-id 2q3nwrw1r0 --stage-name production --export-type oas30 $restAPIId.json

    write-output "👉🏻 Registering API $restAPIId"
    az apic api register -g $resourceGroupName -s $apicResourceName -l $restAPIId".json"

    $apiCustomProperties = @{ 
        "createdDate" = $restAPI.createdDate      
        "rootResourceId" = $restAPI.rootResourceId
        "department" = $restAPI.tags.department
        "dataClassification" = $restAPI.tags.dataClassification
    }
    $apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
    write-output "👉🏻 Updating API $restAPIId metadata"
    az apic api update -g $resourceGroupName -s $apicResourceName --api-id $restAPIId --custom-properties $apiCustomPropertiesJson.replace('"','\"')    

    $openAPIJson = Get-Content -Raw $restAPIId".json"
    $openAPI = $openAPIJson | ConvertFrom-Json
    $version = $openAPI.info.version
    $runtimeUri = $openAPI.servers[0].url

    foreach ($environment in $environments) {
        $environmentName = $environment.title

        # comment one of the following lines to use the real AWS API Gateway or the local file
        # $stageJson =  aws apigateway get-stage --stage-name $environmentName --rest-api-id $restAPIId
        $stageJson = Get-Content -Raw 'get-stage.json'
        
        $stage = $stageJson | ConvertFrom-Json
        $deploymentId = $stage.deploymentId
        $deploymentDescription = $stage.description
        $server = $('{\"runtimeUri\":[\"' + $runtimeUri.replace('{basePath}', $environmentName) + '\"]}')

        $deploymentCustomProperties = @{
            "createdDate" = $stage.createdDate
        }
        $deploymentCustomPropertiesJson = ConvertTo-Json -InputObject $deploymentCustomProperties -Depth 10 -Compress
        write-output "👉🏻 Creating $environmentName deployment for API $restAPIId"
        az apic api deployment create -g $resourceGroupName -s $apicResourceName --deployment-id $environmentName"-"$deploymentId `
                --title $deploymentId --description $deploymentDescription --api-id $restAPIId `
                --environment-id "/workspaces/default/environments/$environmentName" `
                --definition-id "/workspaces/default/apis/$restAPIId/versions/$version/definitions/openapi" `
                --server $server `
                --custom-properties $deploymentCustomPropertiesJson.replace('"','\"')        
    }
}

### 5️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -s $apicResourceName --query "[].{Name:name, Title:title, Kind:kind, ContactEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.